In [ ]:
from netCDF4 import Dataset,num2date,date2num
from numpy import dtype,int16,uint16,float64,int32,uint32,zeros,where,array,NaN,isnan,empty,ones
from pathlib import Path
from pdb import set_trace
from numpy.ma import getmaskarray
from datetime import date,timedelta,datetime
import xarray as xr
import pandas as pd
import numpy as np
import netCDF4 as net
from time import sleep
from timeit import default_timer as timer
from matplotlib import pyplot as plt
import matplotlib.font_manager         as mfm
import matplotlib.ticker               as mtk
import matplotlib.colors               as mcl
import matplotlib.cm                   as mcm
import cartopy.crs                     as ccrs
import cartopy.feature                 as cfeature
import cartopy.mpl.ticker              as ctk
from cartopy.util import add_cyclic_point
from mpl_toolkits.basemap import addcyclic
from copy import deepcopy
import datetime
from datetime import datetime as dt
import time
import sys
from dateutil.relativedelta import relativedelta
import matplotlib.path as mpath
import pickle

data_folder = '../Pickles/'

In [ ]:
def z_masked_overlap(axe, X, Y, Z, source_projection=None):
    """
    for data in projection axe.projection
    find and mask the overlaps (more 1/2 the axe.projection range)

    X, Y either the coordinates in axe.projection or longitudes latitudes
    Z the data
    operation one of 'pcorlor', 'pcolormesh', 'countour', 'countourf'

    if source_projection is a geodetic CRS data is in geodetic coordinates
    and should first be projected in axe.projection

    X, Y are 2D same dimension as Z for contour and contourf
    same dimension as Z or with an extra row and column for pcolor
    and pcolormesh

    return ptx, pty, Z
    """
    if not hasattr(axe, 'projection'):
        return X, Y, Z
    if not isinstance(axe.projection, ccrs.Projection):
        return X, Y, Z
    if len(X.shape) != 2 or len(Y.shape) != 2:
        return X, Y, Z
    if (source_projection is not None and
        isinstance(source_projection, ccrs.Geodetic)):
        transformed_pts = axe.projection.transform_points(
                                   source_projection, X, Y)
        ptx, pty = transformed_pts[..., 0], transformed_pts[..., 1]
    else:
        ptx, pty = X, Y

    with np.errstate(invalid='ignore'):
        # diagonals have one less row and one less columns
        diagonal0_lengths = np.hypot(
                               ptx[1:, 1:] - ptx[:-1, :-1],
                               pty[1:, 1:] - pty[:-1, :-1])
        diagonal1_lengths = np.hypot(
                               ptx[1:, :-1] - ptx[:-1, 1:],
                               pty[1:, :-1] - pty[:-1, 1:])
        to_mask = ((diagonal0_lengths > (
                       abs(axe.projection.x_limits[1]
                           - axe.projection.x_limits[0])) / 2) |
                   np.isnan(diagonal0_lengths) |
                   (diagonal1_lengths > (
                       abs(axe.projection.x_limits[1]
                           - axe.projection.x_limits[0])) / 2) |
                   np.isnan(diagonal1_lengths))
        # TODO check if we need to do something about surrounding vertices
        # add one extra colum and row for contour and contourf
        if (to_mask.shape[0] == Z.shape[0] - 1 and
            to_mask.shape[1] == Z.shape[1] - 1):
            to_mask_extended = np.zeros(Z.shape, dtype=bool)
            to_mask_extended[:-1, :-1] = to_mask
            to_mask_extended[-1, :] = to_mask_extended[-2, :]
            to_mask_extended[:, -1] = to_mask_extended[:, -2]
            to_mask = to_mask_extended
        if np.any(to_mask):
            Z_mask = getattr(Z, 'mask', None)
            to_mask = to_mask if Z_mask is None else to_mask | Z_mask
            Z = np.ma.masked_where(to_mask, Z)

        return ptx, pty, Z


Figure 1

In [ ]:
# Load Sea ice concentration maps
file_map_nsidc_mar = '{}SIC_Mar_1993_2020_CDR.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_nsidc_mar))
file = open(file_map_nsidc_mar, 'rb')
map_nsidc_mar = pickle.load(file)

file_map_nsidc_sep = '{}SIC_Sep_1993_2020_CDR.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_nsidc_sep))
file = open(file_map_nsidc_sep, 'rb')
map_nsidc_sep = pickle.load(file)

Panel (a)

In [ ]:
map_nsidc_sep['SIC'][map_nsidc_sep['SIC'] < 0.01] = np.nan

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})
X, Y, maskedZ = z_masked_overlap(ax, map_nsidc_mar['lon'], map_nsidc_mar['lat'], map_nsidc_mar['SIC'],
                                 source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

min_plot = 0.
max_plot = 1.
cpalette = 'Blues_r'

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='Blues',
                     transform=trans,
                     zorder=2,
                     vmin=min_plot, vmax=max_plot)

ax.contour(map_nsidc_mar['lon'], map_nsidc_mar['lat'], map_nsidc_mar['SIC'], levels=[0.15], colors=['black'], linewidths=2., transform=ccrs.PlateCarree())
ax.contour(map_nsidc_mar['lon'], map_nsidc_mar['lat'], map_nsidc_mar['SIC'], levels=[0.80], colors=['black'], linewidths=2., linestyles="dashed", transform=ccrs.PlateCarree())
cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05)
cbar.ax.tick_params(labelsize=18)
#cbar.set_label(label='         [ ]',fontsize=18,rotation=0)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(a) NSIDC'
ax.set_title(map_title, fontsize=22)
fig.text(0.085, 0.48, 'March', rotation = 90, fontsize=22)
plt.show()

Panel (b)

In [ ]:
map_nsidc_sep['SIC'][map_nsidc_sep['SIC'] < 0.01] = np.nan

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})
X, Y, maskedZ = z_masked_overlap(ax, map_nsidc_sep['lon'], map_nsidc_sep['lat'], map_nsidc_sep['SIC'],
                                 source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

min_plot = 0.
max_plot = 1.
cpalette = 'Blues_r'

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='Blues',
                     transform=trans,
                     zorder=2,
                     vmin=min_plot, vmax=max_plot)

ax.contour(map_nsidc_sep['lon'], map_nsidc_sep['lat'], map_nsidc_sep['SIC'], levels=[0.15], colors=['black'], linewidths=2., transform=ccrs.PlateCarree())
ax.contour(map_nsidc_sep['lon'], map_nsidc_sep['lat'], map_nsidc_sep['SIC'], levels=[0.80], colors=['black'], linewidths=2., linestyles="dashed", transform=ccrs.PlateCarree())
cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05)
cbar.ax.tick_params(labelsize=18)
#cbar.set_label(label='         [ ]',fontsize=18,rotation=0)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(b) NSIDC'
ax.set_title(map_title, fontsize=22)
fig.text(0.085, 0.48, 'September', rotation = 90, fontsize=22)
plt.show()

In [ ]:
# Load Sea ice concentration maps
file_map_grep_mar = '{}DIFFS_map_SIC_Mar_1993_2020_GREPvsNSIDC.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_grep_mar))
file = open(file_map_grep_mar, 'rb')
map_grep_mar = pickle.load(file)

file_map_grep_sep = '{}DIFFS_map_SIC_Sep_1993_2020_GREPvsNSIDC.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_grep_sep))
file = open(file_map_grep_sep, 'rb')
map_grep_sep = pickle.load(file)

lon = map_grep_mar['lon']
lat = map_grep_mar['lat']

Panel (c)

In [ ]:
# Create pcolormesh plot

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})

X, Y, maskedZ = z_masked_overlap(ax, lon, lat, map_grep_mar['SIC'],
                                 source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.OCEAN, color='midnightblue', zorder=0)
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='bwr',
                     transform=trans,
                     zorder=2,
                     vmin=-0.2, vmax=0.2)

cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05, ticks=[-0.20,-0.10,0,0.10,0.20])
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label=' ',fontsize=18)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(c) GREP - NSIDC'
fig.text(0.085, 0.48, 'March', rotation = 90, fontsize=22)
ax.set_title(map_title, fontsize=22)

plt.show()

Panel (f)

In [ ]:
# Create pcolormesh plot

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})

X, Y, maskedZ = z_masked_overlap(ax, lon, lat, map_grep_sep['SIC'],
                                 source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.OCEAN, color='midnightblue', zorder=0)
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='bwr',
                     transform=trans,
                     zorder=2,
                     vmin=-0.2, vmax=0.2)

cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05, ticks=[-0.20,-0.10,0,0.10,0.20])
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label=' ',fontsize=18)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(f) GREP - NSIDC'
fig.text(0.085, 0.48, 'September', rotation = 90, fontsize=22)
ax.set_title(map_title, fontsize=22)

plt.show()

In [ ]:
# Load Sea ice concentration maps
file_map_topaz_mar = '{}DIFFS_map_SIC_Mar_1993_2020_TOPAZvsNSIDC.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_topaz_mar))
file = open(file_map_topaz_mar, 'rb')
map_topaz_mar = pickle.load(file)

file_map_topaz_sep = '{}DIFFS_map_SIC_Sep_1993_2020_TOPAZvsNSIDC.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_topaz_sep))
file = open(file_map_topaz_sep, 'rb')
map_topaz_sep = pickle.load(file)

Panel (d)

In [ ]:
# Create pcolormesh plot

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})

X, Y, maskedZ = z_masked_overlap(ax, lon, lat, map_topaz_mar['SIC'],
                                 source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.OCEAN, color='midnightblue', zorder=0)
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='bwr',
                     transform=trans,
                     zorder=2,
                     vmin=-0.2, vmax=0.2)

cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05, ticks=[-0.20,-0.10,0,0.10,0.20])
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label=' ',fontsize=18)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(d) TOPAZ - NSIDC'
fig.text(0.085, 0.48, 'March', rotation = 90, fontsize=22)
ax.set_title(map_title, fontsize=22)

plt.show()

Panel (g)

In [ ]:
# Create pcolormesh plot

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})

X, Y, maskedZ = z_masked_overlap(ax, lon, lat, map_topaz_sep['SIC'],
                                 source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.OCEAN, color='midnightblue', zorder=0)
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='bwr',
                     transform=trans,
                     zorder=2,
                     vmin=-0.2, vmax=0.2)

cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05, ticks=[-0.20,-0.10,0,0.10,0.20])
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label=' ',fontsize=18)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(g) TOPAZ - NSIDC'
fig.text(0.085, 0.48, 'September', rotation = 90, fontsize=22)
ax.set_title(map_title, fontsize=22)

plt.show()

Panel (e)

In [ ]:
# Load Sea ice concentration maps
file_map_piomas_mar = '{}DIFFS_map_SIC_Mar_1993_2020_PIOMASvsNSIDC.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_piomas_mar))
file = open(file_map_piomas_mar, 'rb')
map_piomas_mar = pickle.load(file)

file_map_piomas_sep = '{}DIFFS_map_SIC_Sep_1993_2020_PIOMASvsNSIDC.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_piomas_sep))
file = open(file_map_piomas_sep, 'rb')
map_piomas_sep = pickle.load(file)

In [ ]:
# Create pcolormesh plot

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})

X, Y, maskedZ = z_masked_overlap(ax, lon, lat, map_piomas_mar['SIC'],
                                 source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.OCEAN, color='midnightblue', zorder=0)
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='bwr',
                     transform=trans,
                     zorder=2,
                     vmin=-0.2, vmax=0.2)

cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05, ticks=[-0.20,-0.10,0,0.10,0.20])
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label=' ',fontsize=18)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(e) PIOMAS - NSIDC'
fig.text(0.085, 0.48, 'March', rotation = 90, fontsize=22)
ax.set_title(map_title, fontsize=22)

plt.show()

In [ ]:
# Create pcolormesh plot

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})

X, Y, maskedZ = z_masked_overlap(ax, lon, lat, map_piomas_sep['SIC'],
                                 source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.OCEAN, color='midnightblue', zorder=0)
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='bwr',
                     transform=trans,
                     zorder=2,
                     vmin=-0.2, vmax=0.2)

cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05, ticks=[-0.20,-0.10,0,0.10,0.20])
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label=' ',fontsize=18)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(h) PIOMAS - NSIDC'
fig.text(0.085, 0.48, 'September', rotation = 90, fontsize=22)
ax.set_title(map_title, fontsize=22)

plt.show()

Figure 6

In [ ]:
# Load Sea ice concentration maps
file_map_grep_mar_sit = '{}SIT_Mar_1993_2020_GREP.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_grep_mar_sit))
file = open(file_map_grep_mar_sit, 'rb')
map_grep_mar_sit = pickle.load(file)

file_map_grep_sep_sit = '{}SIT_Sep_1993_2020_GREP.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_grep_sep_sit))
file = open(file_map_grep_sep_sit, 'rb')
map_grep_sep_sit = pickle.load(file)

Panel (a)

In [ ]:
map_grep_mar_sit['SIT'][map_grep_mar_sit['SIT'] < 0.01] = np.nan

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})

X, Y, maskedZ = z_masked_overlap(ax, lon, lat, map_grep_mar_sit['SIT'], source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

min_plot = 0.
max_plot = 3.5

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='plasma',
                     transform=trans,
                     zorder=2,
                     vmin=min_plot, vmax=max_plot)

ax.contour(lon, lat, map_grep_mar_sit['SIC'], levels=[0.15], colors=['white'], linewidths=2., transform=ccrs.PlateCarree())
ax.contour(lon, lat, map_grep_mar_sit['SIC'], levels=[0.80], colors=['white'], linewidths=2., linestyles="dashed", transform=ccrs.PlateCarree())
cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05)
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label='         [m]',fontsize=18,rotation=0)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(a) GREP'
ax.set_title(map_title, fontsize=22)
fig.text(0.085, 0.48, 'March', rotation = 90, fontsize=22)
plt.show()

Panel (d)

In [ ]:
map_grep_sep_sit['SIT'][map_grep_sep_sit['SIT'] < 0.01] = np.nan

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})

X, Y, maskedZ = z_masked_overlap(ax, lon, lat, map_grep_sep_sit['SIT'], source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

min_plot = 0.
max_plot = 3.5

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='plasma',
                     transform=trans,
                     zorder=2,
                     vmin=min_plot, vmax=max_plot)

ax.contour(lon, lat, map_grep_sep_sit['SIC'], levels=[0.15], colors=['white'], linewidths=2., transform=ccrs.PlateCarree())
ax.contour(lon, lat, map_grep_sep_sit['SIC'], levels=[0.80], colors=['white'], linewidths=2., linestyles="dashed", transform=ccrs.PlateCarree())
cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05)
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label='         [m]',fontsize=18,rotation=0)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(d) GREP'
ax.set_title(map_title, fontsize=22)
fig.text(0.085, 0.48, 'September', rotation = 90, fontsize=22)
plt.show()

In [ ]:
# Load Sea ice concentration maps
file_map_topaz_mar_sit = '{}SIT_Mar_1993_2020_TOPAZ.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_topaz_mar_sit))
file = open(file_map_topaz_mar_sit, 'rb')
map_topaz_mar_sit = pickle.load(file)

file_map_topaz_sep_sit = '{}SIT_Sep_1993_2020_TOPAZ.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_topaz_sep_sit))
file = open(file_map_topaz_sep_sit, 'rb')
map_topaz_sep_sit = pickle.load(file)

Panel (b)

In [ ]:
map_topaz_mar_sit['SIT'][map_topaz_mar_sit['SIT'] < 0.01] = np.nan

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})

X, Y, maskedZ = z_masked_overlap(ax, lon, lat, map_topaz_mar_sit['SIT'], source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

min_plot = 0.
max_plot = 3.5

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='plasma',
                     transform=trans,
                     zorder=2,
                     vmin=min_plot, vmax=max_plot)

ax.contour(lon, lat, map_topaz_mar_sit['SIC'], levels=[0.15], colors=['white'], linewidths=2., transform=ccrs.PlateCarree())
ax.contour(lon, lat, map_topaz_mar_sit['SIC'], levels=[0.80], colors=['white'], linewidths=2., linestyles="dashed", transform=ccrs.PlateCarree())
cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05)
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label='         [m]',fontsize=18,rotation=0)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(b) TOPAZ'
ax.set_title(map_title, fontsize=22)
fig.text(0.085, 0.48, 'March', rotation = 90, fontsize=22)
plt.show()

Panel (e)

In [ ]:
map_topaz_sep_sit['SIT'][map_topaz_sep_sit['SIT'] < 0.01] = np.nan

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})

X, Y, maskedZ = z_masked_overlap(ax, lon, lat, map_topaz_sep_sit['SIT'], source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

min_plot = 0.
max_plot = 3.5

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='plasma',
                     transform=trans,
                     zorder=2,
                     vmin=min_plot, vmax=max_plot)

ax.contour(lon, lat, map_topaz_sep_sit['SIC'], levels=[0.15], colors=['white'], linewidths=2., transform=ccrs.PlateCarree())
ax.contour(lon, lat, map_topaz_sep_sit['SIC'], levels=[0.80], colors=['white'], linewidths=2., linestyles="dashed", transform=ccrs.PlateCarree())
cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05)
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label='         [m]',fontsize=18,rotation=0)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(e) TOPAZ'
ax.set_title(map_title, fontsize=22)
fig.text(0.085, 0.48, 'September', rotation = 90, fontsize=22)
plt.show()

In [ ]:
# Load Sea ice concentration maps
file_map_piomas_mar_sit = '{}SIT_Mar_1993_2020_PIOMAS.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_piomas_mar_sit))
file = open(file_map_piomas_mar_sit, 'rb')
map_piomas_mar_sit = pickle.load(file)

file_map_piomas_sep_sit = '{}SIT_Sep_1993_2020_PIOMAS.pkl'.format(data_folder)
print('Reading: {}'.format(file_map_piomas_sep_sit))
file = open(file_map_piomas_sep_sit, 'rb')
map_piomas_sep_sit = pickle.load(file)

Panel (c)

In [ ]:
map_piomas_mar_sit['SIT'][map_piomas_mar_sit['SIT'] < 0.01] = np.nan

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})

X, Y, maskedZ = z_masked_overlap(ax, lon, lat, map_piomas_mar_sit['SIT'], source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

min_plot = 0.
max_plot = 3.5

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='plasma',
                     transform=trans,
                     zorder=2,
                     vmin=min_plot, vmax=max_plot)

ax.contour(lon, lat, map_piomas_mar_sit['SIC'], levels=[0.15], colors=['white'], linewidths=2., transform=ccrs.PlateCarree())
ax.contour(lon, lat, map_piomas_mar_sit['SIC'], levels=[0.80], colors=['white'], linewidths=2., linestyles="dashed", transform=ccrs.PlateCarree())
cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05)
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label='         [m]',fontsize=18,rotation=0)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(c) PIOMAS'
ax.set_title(map_title, fontsize=22)
fig.text(0.085, 0.48, 'March', rotation = 90, fontsize=22)
plt.show()

Panel (f)

In [ ]:
map_piomas_sep_sit['SIT'][map_piomas_sep_sit['SIT'] < 0.01] = np.nan

proj  = ccrs.NorthPolarStereo()
kwtrans = dict(central_latitude=90, central_longitude=0.)
trans = ccrs.Stereographic(**kwtrans)

fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"projection":proj})

X, Y, maskedZ = z_masked_overlap(ax, lon, lat, map_piomas_sep_sit['SIT'], source_projection=ccrs.Geodetic())

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, color='silver', zorder=3)
ax.add_feature(cfeature.COASTLINE, zorder=3)
ax.gridlines()

min_plot = 0.
max_plot = 3.5

plot = ax.pcolormesh(X, Y, maskedZ,
                     cmap='plasma',
                     transform=trans,
                     zorder=2,
                     vmin=min_plot, vmax=max_plot)

ax.contour(lon, lat, map_piomas_sep_sit['SIC'], levels=[0.15], colors=['white'], linewidths=2., transform=ccrs.PlateCarree())
ax.contour(lon, lat, map_piomas_sep_sit['SIC'], levels=[0.80], colors=['white'], linewidths=2., linestyles="dashed", transform=ccrs.PlateCarree())
cbar = plt.colorbar(plot, ax=ax, shrink=0.5, pad=0.05)
cbar.ax.tick_params(labelsize=18)
cbar.set_label(label='         [m]',fontsize=18,rotation=0)

# Circular boundary of the map
# (see https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html)
theta  = np.linspace(0, 2*np.pi, 100)
center = [0.5, 0.5]
radius =  0.5
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

map_title = '(f) PIOMAS'
ax.set_title(map_title, fontsize=22)
fig.text(0.085, 0.48, 'September', rotation = 90, fontsize=22)
plt.show()